# Commute Weather Pipeline – E2E Inspection
이 노트북은 데이터 수집 → 전처리 → ML 데이터셋 적재 → EDA/시각화 모듈 검증까지 한 번에 확인하기 위한 임시 도구입니다.
작업을 마친 뒤에는 저장소에 커밋하지 않고 삭제하거나 개인 백업용으로만 보관하세요.

## 0. 사전 준비
- `.env`에 KMA API와 S3(LocalStack) 자격 정보가 준비되어 있다고 가정합니다.
- Docker dev 컨테이너 안에서 `pip install jupyterlab` 후 `jupyter lab`으로 접속해 실행하세요.

In [ ]:
from datetime import datetime, timedelta, timezone
from pathlib import Path

import pandas as pd
from IPython.display import display

from src.data.weather_processor import WeatherDataProcessor
from src.utils.config import KMAApiConfig, S3Config
from src.visualization import (
    plot_temperature_trend,
    plot_pm10_distribution,
    plot_uv_intensity,
    plot_comfort_score_timeseries,
    WeatherVisualizationReport,
)
from src.eda import (
    build_overall_profile,
    build_station_profile,
    run_data_quality_checks,
)

kma_config = KMAApiConfig.from_env()
s3_config = S3Config.from_env()
processor = WeatherDataProcessor(kma_config=kma_config, s3_config=s3_config)

print("환경 설정 완료")

## 1. (선택) 최신 기상 데이터 수집 및 S3 저장
`FETCH_NEW_DATA`를 `True`로 바꾸면 KMA API에서 데이터를 받아 S3에 적재합니다.
외부 네트워크가 막혀 있거나 비용이 부담된다면 False로 두고 기존 데이터를 사용하세요.

In [ ]:
FETCH_NEW_DATA = False
TARGET_OFFSET_HOURS = 1

if FETCH_NEW_DATA:
    target_time = datetime.now(tz=timezone.utc) - timedelta(hours=TARGET_OFFSET_HOURS)
    print(f"데이터 수집 시점: {target_time.isoformat()}")

    asos_raw = processor.fetch_weather_data("asos")
    pm10_raw = processor.fetch_weather_data("pm10")
    uv_raw = processor.fetch_weather_data("uv")

    stored = processor.process_and_store_weather_data(
        asos_raw=asos_raw,
        pm10_raw=pm10_raw,
        uv_raw=uv_raw,
        timestamp=target_time,
    )
    print("저장 완료:", stored)
else:
    print("신규 데이터 수집 건너뜀")

## 2. S3 데이터 인벤토리 확인

In [ ]:
inventory = processor.get_data_inventory()
inventory

## 3. ML 정제 데이터셋 로드 및 확인
ML 파이프라인을 거쳐 생성된 parquet 데이터를 불러와 구조를 확인합니다.

In [ ]:
dataset = processor.load_latest_ml_dataset(days_back=7)
if dataset is None or dataset.empty:
    raise RuntimeError("ML 데이터셋을 찾을 수 없습니다. 먼저 process_and_store_weather_data를 실행하세요.")

display(dataset.head())
dataset.info(memory_usage='deep')

### ML 데이터셋 통계 요약

In [ ]:
display(dataset.describe(include='all').transpose())

## 4. 탐색적 데이터 분석 (EDA)

In [ ]:
overall_profile = build_overall_profile(dataset)
station_profile = build_station_profile(dataset, station_id=None)
quality_issues = run_data_quality_checks(dataset)

display(overall_profile["summary"].head())
display(overall_profile["missing"].T)
if not overall_profile["daily_coverage"].empty:
    display(overall_profile["daily_coverage"].tail())

display(station_profile["percentiles"].head())
display(station_profile["comfort_score"])
quality_issues

## 5. 시각화 모듈 검증 – 인라인 플롯

In [ ]:
fig_temp = plot_temperature_trend(dataset)
fig_pm10 = plot_pm10_distribution(dataset)
fig_uv = plot_uv_intensity(dataset)
fig_comfort = plot_comfort_score_timeseries(dataset)

display(fig_temp)
display(fig_pm10)
display(fig_uv)
display(fig_comfort)

## 6. 리포트 저장 흐름 검증

In [ ]:
output_dir = Path("data/visualizations/notebook")
report = WeatherVisualizationReport(output_dir)
generated_files = report.build(dataset)
generated_files

In [ ]:
sorted(output_dir.glob('*'))

## 7. 후처리
- 산출물을 확인했으면 `rm notebooks/tmp_visual_check.ipynb`로 노트북을 삭제하거나 `.gitignore`에 추가해 주세요.
- 저장된 시각화 파일들은 `data/visualizations/notebook/` 경로에 있습니다.